In [1]:
from pathlib import Path
import os
import sys
#import fileOps
#from fileOps import file_operations
#from fileOps import fileLoader
import pandas as pd
from unidecode import unidecode
import unicodedata
import numpy as np
import math as m
import re

## Funciones importantes 

In [3]:
# FUNCIONES IMPORTANTES
 #------------ Remueve acentos 
def remove_accents(input_str):
    # Normalizar el string a su forma combinada
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    # Filtrar y mantener solo los caracteres que no son diacríticos
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
# ----------- Crea data de zonas 
def Zone_data(Hoja,type):
    #excel = "diccionario_Benja/Diccionario_EMTP_DIgSILENT_BVega_v4.xlsx"
    excel = "diccionario_Benja/Zonas_DIgSILENT.xlsx"
    data= pd.read_excel(excel,sheet_name=Hoja)
    if type == "PV":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']  # Reemplaza con los nombres de las columnas que deseas
    elif type == "WP":
         columnas_deseadas = ['Name1','Zona DIgSILENT','Nombre DIgSILENT']     
    elif type == "SG":
        columnas_deseadas = ['Name1', 'Name2','Name3','Zona DIgSILENT','Nombre DIgSILENT']    
    elif type == "PMGD":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']  
    elif type == "CCSS":
        columnas_deseadas = ['Name1', 'Name2','Zona DIgSILENT','Nombre DIgSILENT']    
    elif type =="Cargas":
        columnas_deseadas = ['Carga EMTP','Zona DIgSILENT']   
        
    # Filtra el DataFrame para que contenga solo las columnas deseadas
    dataframe_filtrado = data[columnas_deseadas]
    #return dataframe_filtrado.dropna()   
    return dataframe_filtrado   

def Transformación_MW_MVAR(cadena):
    #cadena_sin_mas = cadena[1:]
    #final= float(cadena_sin_mas)/(10**6)
    final= float(cadena)/(10**6)
    final=round(final,2)
    return (final)
def obtener_V_op(cadena):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', cadena)
    indices_pares = [i for i in range(len(numeros)) if i % 2 == 0]
    valores_pares = [numeros[i] for i in indices_pares]
    lista_V = [round(float(elemento),2) for elemento in valores_pares]
    if lista_V[0] == lista_V[1] == lista_V[2]:
        return lista_V[0]
    else:
        return "desbanceado"
# toma un el string que tiene "V+anglo y me entrega solo el "V y redondeado" 
def separar_numeros(s):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', s)
    if numeros:
        numero_1 = float(numeros[0])  # Convertir a float
        return round(numero_1, 2)
# obtengo cual es la tension nominal a partir del resultado
def analizar_numero(n):
    if 90 <= n + 20 <= 130 or 90 <= n - 20 <= 130:
        return 110
    elif 200 <= n + 20 <= 240 or 200 <= n - 20 <= 240:
        return 220
    elif 46 <= n + 20 <= 86 or 46 <= n - 20 <= 86:
        return 66
    elif 3 <= n + 10 <= 23.2 or 3 <= n - 10 <= 23.2:
        return 13.2    
    else:
        return None  # Si no cae en ninguna categoría


In [2]:
# Aqui coloco el nombre del archivo como string
#archivo =  "SEN 2025 VRE Peak 04072024 DBC 85 GFM New - H1_v4_lf.html"
#archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"
#archivo = "SEN 2025 VRE Peak 85 VRE_g10_10 steps_lf.html"
archivo="SEN 2025 VRE Peak 85 VRE_Inicial_lf.html"
excel = "Datos_nom_v1.xlsx"

def Resultados_Load_Flow(archivo_html,excel):

    Datos_Nominal_extras = pd.read_excel(excel)
    df= pd.read_html(archivo_html)
    columnas = ['Tensión en Bornes [kV]','Potencia Activa [MW]', 'Potencia Reactiva [Mvar]']
    columnas2 = ['Name1','Name2', 'EMTP Load Flow Component']
    df2 = pd.DataFrame(columns=columnas2)
    
    # Crear el DataFrame vacío con las columnas definidas
    df_FINAL = pd.DataFrame(columns=columnas)
    df=pd.DataFrame(df[0])
    # Se procede a transformar el dataframe de html para que este legible 

    #obtengo el nombre las columnas y las reescribo
    fila_cero = df.iloc[0] 
    df.columns =fila_cero.to_list()
    #borro la fila 0 que tenia los nombres de las columnas
    df = df.iloc[1:]
    # reseteo los indices desde 0 y luego borro la primera columna indice
    df=pd.DataFrame(df.reset_index())
    lista_columnas = fila_cero.to_list()
    #  obtengo solo las 5 primeras importantes
    lista_columnas= lista_columnas[:5] 
    df2 = df[lista_columnas[:5]]
    # Crear una lista que solo contenga las centrales y no las cargas 
 
    valores_filtrar = ['PVbus', 'Slack', 'PQbus']
    df_filtrado = df2[df2['Type'].isin(valores_filtrar)]

    df_FINAL['Potencia Activa [MW]']=df_filtrado['P (W)'].apply(Transformación_MW_MVAR)
    df_FINAL['Potencia Reactiva [Mvar]']=df_filtrado['Q (VAR)'].apply(Transformación_MW_MVAR)
    df_FINAL["Tensión en Bornes [kV]"] = df_filtrado["Vabc (kVRMSLL,deg) phasor"].apply(obtener_V_op)
    df_FINAL['Tensión Nominal [kV]'] = df_FINAL["Tensión en Bornes [kV]"].apply(analizar_numero)
   

    #aqui la colummna device la expando para obtener 3 columnas de tal forma que coincida con lo anterior
    nuevo_df = df_filtrado['Device'].str.split('/', expand=True)
    nuevo_df.columns = ['Name1', 'Name2', 'NameLF',"Name4"]

    # Esto entrega la tabla de resultados de todas las centrales del HTML
    df_Resultados = pd.concat([nuevo_df, df_FINAL], axis=1)
    pd.set_option('display.max_rows', 140)

    #---------------------------- saco los acentos 
    df_Resultados['Name1'] = df_Resultados['Name1'].apply(lambda x: unidecode(x))
    df_Resultados['Name2'] = df_Resultados['Name2'].apply(lambda x: unidecode(x))
    df_Resultados = df_Resultados.drop(columns=['Name4']) #no tiene nada util

    ##------------------------Sobre escribo datos nominales------------------------------------
    
    N = len(df_Resultados)
    for fila in range(N):
        Name1= df_Resultados.at[fila,"Name1"]
        Name2= df_Resultados.at[fila,"Name2"]
        NameLF= df_Resultados.at[fila,"NameLF"]
        result = Datos_Nominal_extras[(Datos_Nominal_extras['Name1'] == Name1) &
                                 (Datos_Nominal_extras['Name2'] == Name2) & 
                                 (Datos_Nominal_extras['NameLF'] == NameLF)]
        if len(result)== 1:
            df_Resultados.at[fila,"Tensión Nominal [kV]"] =result["Tensión Nominal [kV]"]
        else: 
            continue 
    

    df_Resultados['Tensión [pu]'] = df_Resultados["Tensión en Bornes [kV]"]/df_Resultados['Tensión Nominal [kV]']
    # -------------------------- cambio orden la columna nombres 
    c=-1
    for name1 in df_Resultados['Name1']:
        c=c+1
        if "Central" in name1 or "Los_Quilos_Juncal" == name1 or "C_ANGOSTURA" == name1:
            df_Resultados.at[c,"Name3"]= name1 
            df_Resultados.at[c,"Name2"]= " " 
    #reordeno columnas 
    nuevo_orden = ['Name1','Name2','Name3','NameLF','Tensión en Bornes [kV]','Potencia Activa [MW]','Potencia Reactiva [Mvar]','Tensión Nominal [kV]','Tensión [pu]']
    #Reordenar las columnas
    df_reordenado = df_Resultados[nuevo_orden]
    return(df_reordenado)
  

Tabla_LF_GEN=Resultados_Load_Flow(archivo,excel)




Tabla_LF_GEN


NameError: name 'Transformación_MW_MVAR' is not defined

## Obteniendo DATA

In [27]:
# version FABI
#carpeta = "Data/Data_14oct"
carpeta = "Data/2030/Data_2030_i0" #"Data/Data_2028_1"
archivos = os.listdir(carpeta)
print(archivos)

df_allSG_csv = [s for s in archivos if "data_allSG" in s]
df_lfToSG_csv = [s for s in archivos if "data_lfToSG" in s]
df_PV_csv = [s for s in archivos if "data_PV" in s]
df_WP_csv = [s for s in archivos if "data_WP" in s]
#df_Load_csv =  [s for s in archivos if "data_Load" in s]

df_allSG  = pd.read_csv(carpeta+"/"+df_allSG_csv[0])
df_lfToSG  = pd.read_csv(carpeta+"/"+df_lfToSG_csv[0])
df_PV  = pd.read_csv(carpeta+"/"+df_PV_csv[0],encoding='latin1')
df_WP  = pd.read_csv(carpeta+"/"+df_WP_csv[0])
#df_Load =  pd.read_csv(carpeta+"/"+df_Load_csv[0])
#data status
df_pv_csv = [s for s in archivos if "uc_pv" in s]
df_wp_csv = [s for s in archivos if "uc_wp" in s]
df_sg_csv = [s for s in archivos if "uc_sg" in s]

df_pv  = pd.read_csv(carpeta+"/"+df_pv_csv[0],encoding='latin1')
df_wp  = pd.read_csv(carpeta+"/"+df_wp_csv[0],encoding='latin1')
df_sg  = pd.read_csv(carpeta+"/"+df_sg_csv[0],encoding='latin1')


#df_PV['Name_PV'] = df_PV['Name_PV'].apply(lambda x: unidecode(x))
## ------------------- correcciones 
## agrego ultimo CCSS al status
df1 = pd.DataFrame({
    'Name1': ["CS_U15"],
    'Name2': ["-"],
    'Name3': ["-"],
    "Status1":	["1"],
    "Status2":	["-"],
    "Status3":	["-"],
    "StatusFinal": 	[1]
})
pd.set_option('display.max_rows', 160)
df_sg = pd.concat([df_sg, df1], axis=0, ignore_index=True)


['data_allSG_2024.11.04 1043 SEN 2030.csv', 'data_lfToSG_2024.11.04 1043 SEN 2030.csv', 'data_LF_2024.11.04 1043 SEN 2030.csv', 'data_PV_2024.11.04 1043 SEN 2030.csv', 'data_WP_2024.11.04 1043 SEN 2030.csv', 'uc_converter_2024.11.04 1043 SEN 2030.csv', 'uc_pv_2024.11.04 1043 SEN 2030.csv', 'uc_sg_2024.11.04 1043 SEN 2030.csv', 'uc_wp_2024.11.04 1043 SEN 2030.csv']


In [8]:
##------------- zonas 
zone_data_WP=Zone_data("Static_WP","WP")
zone_data_PV=Zone_data("Static_PV","PV")
zone_data_SG=Zone_data("SG_UC","SG")

zone_data_PMGD=Zone_data("PMGD","PMGD")
zone_data_CCSS=Zone_data("CCSS","CCSS")
zone_data_Loads = Zone_data("Cargas","Cargas")
zone_data_Loads


,Carga EMTP,Zona DIgSILENT
0,TR_RTOMIC,00-Norte Grande
1,TR_MMH,00-Norte Grande
2,TR_ELTESORO,00-Norte Grande
3,TR_ESPERANZA,00-Norte Grande
4,TR_ANTUCOYA,00-Norte Grande
...,...,...
163,TR_NVA_VALD,10-Araucanía
164,Load2,10-Araucanía
165,TR_LALMEND_N1,04-Enel Distribución
166,TR_ALFALFAL_N1,04-Enel Distribución


# LOAD

In [5]:
datos_load = pd.DataFrame(columns =( "Name","Nombre PowerFactory","Type","EMTP Status","Página PowerFactory","Tensión en Bornes [kV]", "P despachada [MW]", 
                                    "Q despachada [Mvar]","P_set [MW]","Q_set [Mvar]","Tensión Nominal [kV]"))

# con esto encuentro cada cuantas filas hay info de una carga 
filas_no_vacias = df_Load.index[~df_Load['Name_LoadLF'].isna() & (df_Load['Name_LoadLF'] != '')].tolist()     
delta = filas_no_vacias[1] -filas_no_vacias[0]  # resultado cada 6

N= len(df_Load)
c=-1
for i in range(0,N,delta):
    c=c+1
    datos_load.at[c,"Name"]= df_Load.at[i,"Name_LoadLF"]
    datos_load.at[c,"Type"]= df_Load.at[i,"LibType"]
    Pset= float(df_Load.at[i+1,"Value"])
    Qset= float(df_Load.at[i+3,"Value"])
    #las multiplico x3 porque en EMTP se definen por fase
    datos_load.at[c,"P_set [MW]"]= round(3*Pset,2)
    datos_load.at[c,"Q_set [Mvar]"]= round(3*Qset,2)
# agrego zonas 
N = len(zone_data_Loads)         
for i in range(0,N):
    c=-1
    for name1 in datos_load["Name"]:   
        c=c+1
        if name1 == zone_data_Loads.iloc[i,0]:
            datos_load.at[c,"Página PowerFactory"] = zone_data_Loads.iloc[i,1]
        

datos_load

,Name,Nombre PowerFactory,Type,EMTP Status,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],P_set [MW],Q_set [Mvar],Tensión Nominal [kV]
0,TR_LAMPA_220,NaN,PQ load Yg with load-flow (LF),NaN,04-Enel Distribución,NaN,NaN,NaN,45.48,12.56,NaN
1,AJAHUEL_154,NaN,PQ load Yg with load-flow (LF),NaN,06-Troncal_Qui-Cha,NaN,NaN,NaN,68.31,61.66,NaN
2,TR_CHENA_N1,NaN,PQ load Yg with load-flow (LF),NaN,04-Enel Distribución,NaN,NaN,NaN,272.46,68.43,NaN
3,TR_CHENA_N2,NaN,PQ load Yg with load-flow (LF),NaN,04-Enel Distribución,NaN,NaN,NaN,260.43,68.6,NaN
4,TR_AMELIPI_N1,NaN,PQ load Yg with load-flow (LF),NaN,06-Troncal_Qui-Cha,NaN,NaN,NaN,131.4,1.18,NaN
...,...,...,...,...,...,...,...,...,...,...,...
164,L_LLANQUI,NaN,PQ load Yg with load-flow (LF),NaN,10-Araucanía,NaN,NaN,NaN,32.0,7.8,NaN
165,L_PTOMONTT,NaN,PQ load Yg with load-flow (LF),NaN,10-Araucanía,NaN,NaN,NaN,24.0,0.4,NaN
166,MELI_220_66,NaN,PQ load Yg with load-flow (LF),NaN,10-Araucanía,NaN,NaN,NaN,38.0,12.1,NaN
167,Puerto_CMDIC,NaN,PQ load Yg with load-flow (LF),NaN,00-Norte Grande,NaN,NaN,NaN,12.96,6.5,NaN


## <span style="color:Yellow"> DATOS PFV </span>

In [9]:
datos_PV = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_PV["Name1"] =df_pv["Name1"]
datos_PV["Name2"] =df_pv["Name2"]   
datos_PV["EMTP Status"] = df_pv["StatusFinal"]

#Elimina los PMGD
datos_PV = datos_PV[~datos_PV['Name1'].str.contains('PMGD', na=False)]
datos_PV = datos_PV[~datos_PV['Name1'].str.contains('OG', na=False)]
datos_PV = datos_PV.reset_index(drop=True)
datos_PV["Type"] = len(datos_PV)*["Parque Fotovoltaico"]

## hay PFV nuevos sin name2, por lo que quienes no tengan name2 repito el name1 
c=-1
for name2  in datos_PV["Name2"]:
    c=c+1
    if name2 == "-":
        name1 = datos_PV.at[c,'Name1'] 
        datos_PV.at[c,'Name2']= name1


c=-1
for name2 in datos_PV["Name2"]:
    c=c+1
    indice= df_PV.index[df_PV['Name_PV'] ==name2].tolist()[0] 
    Spv= df_PV.at[indice+7,"Value"]
    Vnom = df_PV.at[indice+3,"Value"]
    Npv =df_PV.at[indice,"Value"]
    Npv_serv = df_PV.at[indice+17,"Value"]
    Snom = Spv*Npv
    Sbase = Spv*Npv_serv
    Qset = df_PV.at[indice+19,"Value"]*Sbase
    Pset = df_PV.at[indice+21,"Value"]*Sbase
    
    #--------------------------------------------------------------
    datos_PV.at[c,'Nº_unid']= Npv
    datos_PV.at[c,'Nº_unid_serv']= Npv_serv
    datos_PV.at[c,'Tensión Nominal [kV]']=Vnom
    datos_PV.at[c,"Potencia Nominal [MVA]"] = Snom
    datos_PV.at[c,"Sbase [MVA]"] = Sbase
    datos_PV.at[c,"P_set [MW]"] = Pset
    datos_PV.at[c,"Q_set [Mvar]"] = Qset


# agrego Nombre de PowerFactory y su Pagina    
N = len(zone_data_PV)         
for i in range(0,N):
    c=-1
    for name1 in datos_PV["Name1"]:   
        c=c+1
        if name1 == zone_data_PV.iloc[i,0]:
            datos_PV.at[c,"Página PowerFactory"] = zone_data_PV.iloc[i,2]
            datos_PV.at[c,'Nombre PowerFactory'] = zone_data_PV.iloc[i,3]
        else:
            continue     
datos_PV['Name1'] = datos_PV['Name1'].apply(lambda x: unidecode(x))

datos_PV['Name2'] = datos_PV['Name2'].apply(lambda x: unidecode(x))        

#---------- arreglando paginas faltantes no me reconoce coquimo
datos_PV.at[0,"Página PowerFactory"]= "02-Coquimbo"

datos_PV  


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,110.0022,67.871357,-16.104322,NaN,220.0,110.0022,66.0,66.0
1,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,94.38,58.23246,-18.876,NaN,220.0,94.38,26.0,26.0
2,PFV_EL_ROMERO,PFV_EL_ROMERO,NaN,PFV EL ROMERO,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,206.8891,127.650575,-41.37782,NaN,220.0,206.8891,190.0,190.0
3,PFV_VALLE_ESCONDIDO,PFV_VALLE_ESCONDIDO,NaN,PFV VALLE ESCONDIDO,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,106.688,65.826496,-10.6688,NaN,220.0,106.688,64.0,64.0
4,PFV_SAN_ANDRES,PFV_SAN_ANDRES,NaN,PFV SAN ANDRES,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,50.575,13.564215,0.0,NaN,220.0,50.575,35.0,35.0
5,PFV_LLANO_DE_LLAMPOS,PFV_LLANO_DE_LLAMPOS,NaN,PFV LLANO DE LLAMPOS,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,110.022,67.883574,-22.0044,NaN,220.0,110.022,66.0,66.0
6,PFV_Pampa_Solar_Norte,PFV_Pampa_Solar_Norte,NaN,PFV PAMPA SOLAR NORTE,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,70.014,43.198638,-6.30126,NaN,220.0,70.014,42.0,42.0
7,PFV_GUANCHOI,PFV_GUANCHOI,NaN,PFV GUANCHOI 01 A,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,50.01,38.40768,0.0,NaN,220.0,50.01,30.0,30.0
8,PFV_D_ALMAGRO_SUR,PFV_DIEGO_DE_ALMAGRO_SUR,NaN,PFV Diego de Almagro Sur,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,211.6,162.10676,0.0,NaN,220.0,211.6,46.0,46.0
9,PFV_CAMPOS_DEL_SOL,PFV_CAMPOS_DEL_SOL,NaN,PFV Campos de Sol 1 64.7 MVA,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,408.415,312.886731,-24.545742,NaN,220.0,408.415,245.0,245.0


# Data PMGD

In [10]:
datos_PMGD = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_PMGD["Name1"] =df_pv["Name1"]
datos_PMGD["Name2"] =df_pv["Name2"]   
datos_PMGD["EMTP Status"] = df_pv["StatusFinal"]

#EConversa solo los que digan PMGD
datos_PMGD = datos_PMGD[datos_PMGD['Name1'].str.contains('PMGD', na=False)]
datos_PMGD = datos_PMGD.reset_index(drop=True)
datos_PMGD["Type"] = len(datos_PMGD)*["PMGD"]

c=-1
for name1 in datos_PMGD["Name1"]:
    
    c=c+1
    indice= df_PV.index[df_PV['Name_PV'] ==name1].tolist()[0] 
    Spv= df_PV.at[indice+7,"Value"]
    Vnom = df_PV.at[indice+3,"Value"]
    Npv =df_PV.at[indice,"Value"]
    Npv_serv = df_PV.at[indice+17,"Value"]
    Snom = Spv*Npv
    Sbase = Spv*Npv_serv
    Qset = df_PV.at[indice+19,"Value"]
    Pset = df_PV.at[indice+21,"Value"]
    
    #--------------------------------------------------------------
    datos_PMGD.at[c,'Nº_unid']= int(Npv)
    datos_PMGD.at[c,'Nº_unid_serv']= int(Npv_serv)
    datos_PMGD.at[c,'Tensión Nominal [kV]']=Vnom
    datos_PMGD.at[c,"Potencia Nominal [MVA]"] = Snom
    datos_PMGD.at[c,"Sbase [MVA]"] = Sbase
    datos_PMGD.at[c,"P_set [MW]"] = Pset*Sbase
    datos_PMGD.at[c,"Q_set [Mvar]"] = Qset*Sbase


# agrego Nombre de PowerFactory y su Pagina    
N = len(zone_data_PMGD)         
for i in range(0,N):
    c=-1
    for name1 in datos_PMGD["Name1"]:   
        c=c+1
        if name1 == zone_data_PMGD.iloc[i,0]:
            datos_PMGD.at[c,"Página PowerFactory"] = zone_data_PMGD.iloc[i,2]
            datos_PMGD.at[c,'Nombre PowerFactory'] = zone_data_PMGD.iloc[i,3]
        else:
            continue     
#datos_PMGD['Name1'] = datos_PV['Name1'].apply(lambda x: unidecode(x))

#datos_PMGD['Name2'] = datos_PV['Name2'].apply(lambda x: unidecode(x))        
#----------------------------------------------------------
 #estos nose porque no se escribia, asique los puse yo
datos_PMGD.at[0,"Página PowerFactory"]  = "00-Norte Grande"
datos_PMGD.at[1,"Página PowerFactory"]  = "00-Norte Grande"



datos_PMGD

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PMGD_PFV_Cardones110,-,NaN,NaN,PMGD,1,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,138.0,37.0116,0.0,NaN,220.0,138.0,138,138
1,PMGD_PFV_DAlmagro110,-,NaN,NaN,PMGD,1,NaN,00-Norte Grande,NaN,NaN,NaN,NaN,41.0,27.8349,0.0,NaN,110.0,41.0,41,41
2,PMGD_PFV_Maitencillo110,-,NaN,NaN,PMGD,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,38.0,8.2308,0.0,NaN,220.0,38.0,38,38
3,PMGD_PFV_Quillota110,-,NaN,NaN,PMGD,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,52.0,34.4396,0.0,NaN,220.0,52.0,52,52
4,PMGD_PFV_ElPenon110,-,NaN,NaN,PMGD,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,215.0,61.619,0.0,NaN,220.0,215.0,215,215
5,PMGD_PFV_LVilos220,-,NaN,NaN,PMGD,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,116.0,21.46,0.0,NaN,220.0,116.0,116,116
6,PMGD_PFV_PAzucar110,-,NaN,NaN,PMGD,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,96.0,27.5136,0.0,NaN,220.0,96.0,96,96
7,PMGD_PFV_ASanta220,-,NaN,NaN,PMGD,1,NaN,03-Chilquinta-Aconcagua,NaN,NaN,NaN,NaN,121.0,0.121,0.0,NaN,220.0,121.0,121,121
8,PMGD_PFV_LVegas110,-,NaN,NaN,PMGD,1,NaN,03-Chilquinta-Aconcagua,NaN,NaN,NaN,NaN,141.0,38.1123,0.0,NaN,110.0,141.0,141,141
9,PMGD_PFV_Batuco110,-,NaN,NaN,PMGD,1,NaN,04-Enel Distribución,NaN,NaN,NaN,NaN,62.0,39.7482,0.0,NaN,220.0,62.0,62,62


# Data PFV Adicionales 

In [16]:
datos_PV_OG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))

datos_PV_OG["Name1"] =df_pv["Name1"]
datos_PV_OG["Name2"] =df_pv["Name2"]   
datos_PV_OG["EMTP Status"] = df_pv["StatusFinal"]

#EConversa solo los que digan PMGD
datos_PV_OG = datos_PV_OG[datos_PV_OG['Name1'].str.contains('OG', na=False)]
datos_PV_OG = datos_PV_OG.reset_index(drop=True)
datos_PV_OG["Type"] = len(datos_PV_OG)*["Parque Fotovoltaico"]

c=-1
for name1 in datos_PV_OG["Name1"]:
    if name1 == "PFV_Lascar_OG":
        name1 = "PFV_Lascar"
    
    c=c+1
    print(name1)
    indice= df_PV.index[df_PV['Name_PV'] ==name1].tolist()[0] 
    Spv= df_PV.at[indice+7,"Value"]
    Vnom = df_PV.at[indice+3,"Value"]
    Npv =df_PV.at[indice,"Value"]
    Npv_serv = df_PV.at[indice+17,"Value"]
    Snom = Spv*Npv
    Sbase = Spv*Npv_serv
    Qset = df_PV.at[indice+19,"Value"]
    Pset = df_PV.at[indice+21,"Value"]
    
    #--------------------------------------------------------------
    datos_PV_OG.at[c,'Nº_unid']= int(Npv)
    datos_PV_OG.at[c,'Nº_unid_serv']= int(Npv_serv)
    datos_PV_OG.at[c,'Tensión Nominal [kV]']=Vnom
    datos_PV_OG.at[c,"Potencia Nominal [MVA]"] = Snom
    datos_PV_OG.at[c,"Sbase [MVA]"] = Sbase
    datos_PV_OG.at[c,"P_set [MW]"] = Pset*Sbase
    datos_PV_OG.at[c,"Q_set [Mvar]"] = Qset*Sbase
# agrego Nombre de PowerFactory y su Pagina    
N = len(zone_data_PV)         
for i in range(0,N):
    c=-1
    for name1 in datos_PV_OG["Name1"]:   
        c=c+1
        if name1 == zone_data_PV.iloc[i,0]:
            datos_PV_OG.at[c,"Página PowerFactory"] = zone_data_PV.iloc[i,2]
            #datos_PV.at[c,'Nombre PowerFactory'] = zone_data_PV.iloc[i,3]
        else:
            continue 
datos_PV_OG

PFV_Dona_Antonia_OG
PFV_Don_Humberto_OG
PFV_Tamarico_OG
PFV_Libertad_II_OG
PFV_Libertad_III_OG
PFV_Leyda_OG
PFV_Central_DDA_OG
PFV_Sol_de_Vallenar_OG
PFV_Peldehue_Solar_OG
PFV_Cachiyuyo_OG
PFV_Lascar
PFV_Las_Salinas_Etapa2_OG
PFV_Las_Salinas_Etapa3_OG
PFV_Las_Salinas_Etapa4_OG
PFV_Las_Salinas_Etapa5_OG
PFV_Las_Salinas_Etapa6_OG
PFV_Aurora_Solar_OG
PFV_Gabriela_OG
PFV_Andes_III_Etapa1_OG
PFV_Victor_Jara_OG
PFV_Gran_Teno_OG
PFV_Tamango_OG
PMG_Venezia_Solar_OG
PFV_Ortega_Negrete_OG
PFV_Solar_Toro_OG
PFV_Zaldívar_OG
PFV_Socompa_OG
PFV_Tirana_Oeste_OG


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_Dona_Antonia_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,82.0,23.5012,0.0,NaN,220.0,82.0,82,82
1,PFV_Don_Humberto_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,NaN,NaN,NaN,NaN,81.0,21.8943,0.0,NaN,110.0,81.0,81,81
2,PFV_Tamarico_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,165.0,35.739,0.0,NaN,220.0,165.0,165,165
3,PFV_Libertad_II_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,136.0,29.4576,0.0,NaN,220.0,136.0,136,136
4,PFV_Libertad_III_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,136.0,29.4576,0.0,NaN,220.0,136.0,136,136
5,PFV_Leyda_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,03-Chilquinta-Aconcagua,NaN,NaN,NaN,NaN,95.0,9.5,0.0,NaN,220.0,95.0,95,95
6,PFV_Central_DDA_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,293.0,77.5864,0.0,NaN,220.0,293.0,293,293
7,PFV_Sol_de_Vallenar_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,123.0,32.5704,0.0,NaN,220.0,123.0,123,123
8,PFV_Peldehue_Solar_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,06-Troncal_Qui-Cha,NaN,NaN,NaN,NaN,120.0,32.712,0.0,NaN,220.0,120.0,120,120
9,PFV_Cachiyuyo_OG,-,NaN,NaN,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,53.0,11.4798,0.0,NaN,220.0,53.0,53,53


# Data WP


In [17]:
datos_WP = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_WP["Name1"] =df_wp["Name1"]
datos_WP["Name2"] =df_wp["Name2"]   
datos_WP["EMTP Status"] = df_wp["StatusFinal"]
datos_WP["Type"] = len(datos_WP)*["Parque Eólico"]

c=-1
for name1 in datos_WP["Name1"]:
    c=c+1
    indice= df_WP.index[df_WP['Name_WP'] ==name1].tolist()[0]

    Swp= df_WP.at[indice+7,"Value"]
    Vnom = df_WP.at[indice+3,"Value"]
    Nwp =df_WP.at[indice,"Value"]
    
    Snom = Swp*Nwp
    
    
    if name1 != "PE_Taltal":
        Pset = df_WP.at[indice+20,"Value"]
        Qset = df_WP.at[indice+19,"Value"]
        Nwp_serv = df_WP.at[indice+17,"Value"]
        Sbase = Swp*Nwp_serv
    else :
        Pset =0
        # Qset = df_WP.at[indice+18,"Value"]   
        Nwp_serv = df_WP.at[indice+16,"Value"]
        Sbase = Spv*Npv_serv  

    #--------------------------------------------------------------
    datos_WP.at[c,'Nº_unid']= int(Nwp)
    datos_WP.at[c,'Nº_unid_serv']= int(Nwp_serv)
    datos_WP.at[c,'Tensión Nominal [kV]']=Vnom
    datos_WP.at[c,"Potencia Nominal [MVA]"] = Snom
    datos_WP.at[c,"Sbase [MVA]"] = Sbase
    datos_WP.at[c,"P_set [MW]"] = Pset*Sbase
    datos_WP.at[c,"Q_set [Mvar]"] = Qset*Sbase

#----------------------------------------------------------------------------
c=-1 
N = len(zone_data_WP)
Nstatus= len(df_wp)
for name in datos_WP["Name1"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name == zone_data_WP.iloc[i,0]:
            datos_WP.at[c,"Página PowerFactory"] = zone_data_WP.iloc[i,1]
            datos_WP.at[c,'Nombre PowerFactory'] = zone_data_WP.iloc[i,2]
        else:
            continue                    

datos_WP      

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PE_El_Arrayan,-,NaN,PE EL ARRAYAN (U1-U50),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,116.69,33.443354,0.0,NaN,220.0,116.69,70,70
1,PE_Pta_Palmeras,-,NaN,PE PUNTA PALMERAS (U1-U15),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,45.009,37.402479,-6.75135,NaN,220.0,45.009,27,27
2,PE_Talinay_O_P,-,NaN,PE TALINAY ORIENTE (U1-U30),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,161.699,67.3468,0.0,NaN,220.0,161.699,97,97
3,PE_Monte_Redondo,-,NaN,PE MONTE REDONDO (U1-U24),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,48.343,16.90338,0.0,NaN,220.0,48.343,29,29
4,PE_CANELA_II,-,NaN,PE CANELA II (U1-U40),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,63.346,18.60372,-8.335,NaN,220.0,63.346,38,38
5,PE_LOS_CURUROS,-,NaN,PE LOS CURUROS (U1-U57),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,115.023,75.11502,0.0,NaN,220.0,115.023,69,69
6,PE_PUNTA_SIERRA,-,NaN,PE PUNTA SIERRA (U1-U32),Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,85.017,1.30026,-9.3352,NaN,220.0,85.017,51,51
7,PE_Cabo_Leones_I,-,NaN,PE CABO LEONES I TIPO 1 (2 [MW])_A,Parque Eólico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,115.5,25.0173,0.0,NaN,220.0,115.5,55,55
8,PE_La_Estrella,-,NaN,PE LA ESTRELLA,Parque Eólico,1,NaN,NaN,NaN,NaN,NaN,NaN,53.344,45.009,0.0,NaN,110.0,53.344,32,32
9,PE_San_Juan_I_II,-,NaN,PE SAN JUAN (U1-U56)_A,Parque Eólico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,186.704,53.509366,0.0,NaN,220.0,186.704,112,112


# Datos SG

In [18]:
datos_SG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_SG["Name1"] =df_sg["Name1"]
datos_SG["Name2"] =df_sg["Name2"]
datos_SG["Name3"] =df_sg["Name3"]
datos_SG["EMTP Status"] = df_sg["StatusFinal"]
##----------- elimino los OG

datos_SG = datos_SG[~datos_SG['Name1'].str.contains('OG', na=False)]
datos_SG = datos_SG[~datos_SG['Name1'].str.contains('CCSS', na=False)]
datos_SG = datos_SG[~datos_SG['Name1'].str.contains('SM3', na=False)]
datos_SG = datos_SG.reset_index(drop=True)
datos_SG["Type"] = len(datos_SG)*["Generador Sincrónico"]


##-------------------------------------------------------------------

pd.set_option('display.max_rows', 151)


c=-1
l_aux= []  
for name3 in datos_SG["Name3"]:
    l_aux.append(name3)
    
    c=c+1
    try:
        indice = df_lfToSG.index[df_lfToSG['AssociateSG'] == name3].tolist()[0]
        resultado = "sí"
    except IndexError:
        resultado = "no"
        print(name3)
    datos_SG.at[c,"EMTP Load Flow Component"]= df_lfToSG.at[indice,"Name_of_LF"] #Lf
    datos_SG.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
    datos_SG.at[c,"V_set [kV]"]= df_lfToSG.at[indice+1,"Value"]   # V_set
    datos_SG.at[c,"P_set [MW]"]= df_lfToSG.at[indice+2,"Value"]   # P_set
    datos_SG.at[c,"Q_set [Mvar]"]= df_lfToSG.at[indice+3,"Value"] # Q_set
     
   

 
 
 

# agrego data nominal    

N = len(df_allSG)
c=-1
for name3 in datos_SG["Name3"]:
    c=c+1
    for i in range(0,N):
        if name3 == df_allSG.at[i,"Name_of_SG"]:
             datos_SG.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
             datos_SG.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
             datos_SG.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set
            
# agrego data PF
N = len(zone_data_SG)
c=-1
for name3 in datos_SG["Name3"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name3 == zone_data_SG.iloc[i,2]:
            datos_SG.at[c,"Página PowerFactory"] = zone_data_SG.iloc[i,3]
            datos_SG.at[c,'Nombre PowerFactory'] = zone_data_SG.iloc[i,4]
        else:
            continue
Nsg = len(datos_SG)
datos_SG["Type"] = ["Generador Sincrónico"]*Nsg          
datos_SG["Nº_unid_serv"] = ["1"]*Nsg   
datos_SG["Nº_unid"] = ["1"]*Nsg         
#pd.set_option('display.max_rows', 151)
datos_SG     


    


-


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,Central_San_Isidro,San_Isidro_U2_TV,San_Isidro_U2_TV,TER SAN ISIDRO II CC1-TV,Generador Sincrónico,1,LF_SISIDRO_U2_TV,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,163.0,64,480,13.2,13.2,163.0,1,1
1,Central_San_Isidro,San_Isidro_U2_TG,San_Isidro_U2_TG,TER SAN ISIDRO II CC1-TG,Generador Sincrónico,1,LF_SISIDRO_U2_TG,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,305.0,105,480,15.0,15.0,305.0,1,1
2,Central_San_Isidro,San_Isidro_U1_TG,San_Isidro_U1_TG,TER SAN ISIDRO CC1-TG,Generador Sincrónico,0,LF_SISIDRO_U1_TG,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,283.59,113,480,13.8,13.8,283.59,1,1
3,Central_San_Isidro,San_Isidro_U1_TV,San_Isidro_U1_TV,TER SAN ISIDRO CC1-TV,Generador Sincrónico,0,LF_SISIDRO_U1_TV,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,162.412,110,480,15,15.0,162.412,1,1
4,Central_Nehuenco,Nehuenco_U2_TG,Nehuenco_U2_TG,TER NEHUENCO II CC1-TG,Generador Sincrónico,1,LF_NEHU_U2_TG,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,300.0,110,480,15.75,15.75,300.0,1,1
5,Central_Nehuenco,Nehuenco_U2_TV,Nehuenco_U2_TV,TER NEHUENCO II CC1-TV,Generador Sincrónico,1,LF_NEHU_U2_TV,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,164.0,85,480,15.75,15.75,164.0,1,1
6,Central_Nehuenco,Nehuenco_U1_TV,Nehuenco_U1_TV,TER NEHUENCO CC1-TV,Generador Sincrónico,0,LF_NEHU_U1_TV,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,175.0,82.8,480,10.8962,10.5,175.0,1,1
7,Central_Nehuenco,Nehuenco_U1_TG,Nehuenco_U1_TG,TER NEHUENCO CC1-TG,Generador Sincrónico,0,LF_NEHU_U1_TG,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,273.0,165.7,480,15.8,15.75,273.0,1,1
8,Central_Nehuenco,Nehuenco_U3,Nehuenco_U3,TER NEHUENCO 9B U1,Generador Sincrónico,0,LF_NEHU_U3,06-Troncal_Qui-Cha,NaN,NaN,NaN,PV,111.625,0,480,10.5,10.5,111.625,1,1
9,Central_Hornitos,Hornitos_U1,Hornitos_U1,HP HORNITOS U1,Generador Sincrónico,1,LF_HORNIT_U1,03-Chilquinta-Aconcagua,NaN,NaN,NaN,PQ,59.8,22.6,-7.9,11,11.0,59.8,1,1


# DATOS SG OG

In [25]:
datos_SG_OG = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_SG_OG["Name1"] =df_sg["Name1"]
datos_SG_OG["Name2"] =df_sg["Name2"]
datos_SG_OG["Name3"] =df_sg["Name3"]
datos_SG_OG["EMTP Status"] = df_sg["StatusFinal"]
##----------- elimino los OG
datos_SG_OG = datos_SG_OG[~datos_SG_OG['Name1'].str.contains('CCSS', na=False)]
datos_SG = datos_SG[~datos_SG['Name1'].str.contains('SM3', na=False)]
datos_SG_OG = datos_SG_OG[datos_SG_OG['Name1'].str.contains('OG', na=False)]


datos_SG_OG = datos_SG_OG.reset_index(drop=True)
datos_SG_OG["Type"] = len(datos_SG_OG)*["Generador Sincrónico"]

df_lfToSGOG = pd.read_excel('Plantas.xlsx',sheet_name="SGOG" )
##-------------------------------------------------------------------

c=-1
l_aux= []  
for name1 in datos_SG_OG["Name1"]:
    l_aux.append(name3)
    
    c=c+1
    try:
        indice = df_lfToSGOG.index[df_lfToSGOG['AssociateSG'] == name1].tolist()[0]
        resultado = "sí"
    except IndexError:
        resultado = "no"
        print(name1)
    datos_SG_OG.at[c,"EMTP Load Flow Component"]= df_lfToSG.at[indice,"Name_of_LF"] #Lf
    datos_SG_OG.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
    datos_SG_OG.at[c,"V_set [kV]"]= df_lfToSG.at[indice+1,"Value"]   # V_set
    datos_SG_OG.at[c,"P_set [MW]"]= df_lfToSG.at[indice+2,"Value"]   # P_set
    datos_SG_OG.at[c,"Q_set [Mvar]"]= df_lfToSG.at[indice+3,"Value"] # Q_set
     
# agrego data nominal    

N = len(df_allSG)
c=-1
for name1 in datos_SG_OG["Name1"]:
    c=c+1
    for i in range(0,N):
        if name1 == df_allSG.at[i,"Name_of_SG"]:
             datos_SG_OG.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
             datos_SG_OG.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
             datos_SG_OG.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set
            
# agrego data PF
N = len(zone_data_SG)
c=-1
for name1 in datos_SG_OG["Name1"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name3 == zone_data_SG.iloc[i,2]:
            datos_SG_OG.at[c,"Página PowerFactory"] = zone_data_SG.iloc[i,3]
            datos_SG_OG.at[c,'Nombre PowerFactory'] = zone_data_SG.iloc[i,4]
        else:
            continue
Nsg = len(datos_SG_OG)
datos_SG_OG["Type"] = ["Generador Sincrónico"]*Nsg          
datos_SG_OG["Nº_unid_serv"] = ["1"]*Nsg   
datos_SG_OG["Nº_unid"] = ["1"]*Nsg         
#pd.set_option('display.max_rows', 151)   

pd.set_option('display.max_rows', 151)




datos_SG_OG      

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,TER_DONA_LUZMA_OG,-,-,NaN,Generador Sincrónico,0,LF_CARDO_U1,NaN,NaN,NaN,NaN,PQ,40.0,0,0,15.75,13.8,40.0,1,1
1,HP_TRUPAN_OG,-,-,NaN,Generador Sincrónico,0,LF_NVENTA,NaN,NaN,NaN,NaN,PV,20.0,80,480,18.18,13.8,20.0,1,1
2,HP_DOS_VALLES_AMP_OG,-,-,NaN,Generador Sincrónico,0,LF_CAMPI,NaN,NaN,NaN,NaN,PV,5.0,270,480,18,13.8,5.0,1,1
3,HP_LOS_CONDORES_OG,-,-,NaN,Generador Sincrónico,0,LF_VENTA_U1,NaN,NaN,NaN,NaN,PV,162.0,70,480,13.524,13.8,162.0,1,1
4,HP_NUBLE_OG,-,-,NaN,Generador Sincrónico,0,LF_VENTA_U2,NaN,NaN,NaN,NaN,PV,136.0,88,480,18,13.8,136.0,1,1
5,HP_LOS_LAGOS_OG,-,-,NaN,Generador Sincrónico,0,LF_TALT_U1,NaN,NaN,NaN,NaN,PV,54.0,0,480,15,13.8,54.0,1,1


# Datos CCSS

In [30]:
datos_CCSS = pd.DataFrame(columns=("Name1","Name2","Name3","Nombre PowerFactory","Type","EMTP Status","EMTP Load Flow Component","Página PowerFactory","Tensión en Bornes [kV]",
                                   "P despachada [MW]","Q despachada [Mvar]", "Tipo de Barra Flujo de Carga","Sbase [MVA]","P_set [MW]","Q_set [Mvar]","V_set [kV]","Tensión Nominal [kV]", "Potencia Nominal [MVA]"
                                   ,'Nº_unid_serv','Nº_unid'))
datos_CCSS["Name1"] =df_sg["Name1"]
datos_CCSS["Name2"] =df_sg["Name2"]
datos_CCSS["Name3"] =df_sg["Name3"]
datos_CCSS["EMTP Status"] = df_sg["StatusFinal"]
##----------- elimino los OG
#datos_CCSS2= datos_CCSS[datos_CCSS['Name1'].str.contains('SM3', na=False)]
datos_CCSS2 = datos_CCSS[datos_CCSS['Name1'].str.contains('CS_U15', na=False)]
datos_CCSS = datos_CCSS[datos_CCSS['Name1'].str.contains('CCSS', na=False)]
datos_CCSS = pd.concat([datos_CCSS, datos_CCSS2], axis=0, ignore_index=True)
#datos_CCSS = pd.concat([datos_CCSS, datos_CCSS2], axis=0, ignore_index=True)

datos_CCSS = datos_CCSS.reset_index(drop=True)
datos_CCSS["Type"] = len(datos_CCSS)*["Condensador Sincronico"]

df_lfToSGCCSS = pd.read_excel('Plantas.xlsx',sheet_name='CCSS')
print(datos_CCSS)
##-------------------------------------------------------------------



c=-1
l_aux= []  
for name1 in datos_CCSS["Name1"]:
    l_aux.append(name3)
    
    c=c+1
    try:
        indice = df_lfToSGCCSS.index[df_lfToSGCCSS ['AssociateSG'] == name1].tolist()[0]
        resultado = "sí"
    except IndexError:
        resultado = "no"
        print(name1)
    datos_CCSS.at[c,"EMTP Load Flow Component"]= df_lfToSGCCSS.at[indice,"Name_of_LF"] #Lf
    datos_CCSS.at[c,"Tipo de Barra Flujo de Carga"]= df_lfToSG.at[indice,"Value"] #tipo de barra
    datos_CCSS.at[c,"V_set [kV]"]= df_lfToSGCCSS.at[indice+1,"Value"]   # V_set
    datos_CCSS.at[c,"P_set [MW]"]= df_lfToSGCCSS.at[indice+2,"Value"]   # P_set
    datos_CCSS.at[c,"Q_set [Mvar]"]= df_lfToSGCCSS.at[indice+3,"Value"] # Q_set
# agrego data nominal    

N = len(df_allSG)
c=-1
for name1 in datos_CCSS["Name1"]:
    c=c+1
    for i in range(0,N):
        if name1 == df_allSG.at[i,"Name_of_SG"]:
             datos_CCSS.at[c,"Tensión Nominal [kV]"]= df_allSG.at[i+3,"Value"]
             datos_CCSS.at[c,"Potencia Nominal [MVA]"]= df_allSG.at[i+4,"Value"]
             datos_CCSS.at[c,"Sbase [MVA]"] = df_allSG.at[i+4,"Value"]   # V_set
                
            
# agrego data PF
N = len(zone_data_CCSS)
c=-1
for name1 in datos_CCSS["Name1"]:
    c=c+1
    #agrego zona y pagina PF
    for i in range(0,N):
        if name1 == zone_data_CCSS.iloc[i,0] :
            print(name1)
            datos_CCSS.at[c,"Página PowerFactory"] = zone_data_CCSS.iloc[i,2]
            datos_CCSS.at[c,'Nombre PowerFactory'] = zone_data_CCSS.iloc[i,3]
        else:
            continue     
Nsg = len(datos_CCSS)
datos_CCSS["Nº_unid_serv"] = ["1"]*Nsg   
datos_CCSS["Nº_unid"] = ["1"]*Nsg         
#pd.set_option('display.max_rows', 151)   
##------------------------------------

datos_CCSS.loc[datos_CCSS['Name1'] == 'CS_U15', 'Sbase [MVA]'] = 147.1

#         
datos_CCSS

                       Name1 Name2 Name3 Nombre PowerFactory  \
0      CCSS_AMARIA_TRANSELEC     -     -                 NaN   
1  CCSS_LIKANANTAI_TRANSELEC     -     -                 NaN   
2         CCSS_ILLAPA_ALUPAR     -     -                 NaN   
3         CCSS_AMARIA_ALUPAR     -     -                 NaN   
4                     CS_U15     -     -                 NaN   

                     Type  EMTP Status EMTP Load Flow Component  \
0  Condensador Sincronico            1                      NaN   
1  Condensador Sincronico            1                      NaN   
2  Condensador Sincronico            1                      NaN   
3  Condensador Sincronico            1                      NaN   
4  Condensador Sincronico            1                      NaN   

  Página PowerFactory Tensión en Bornes [kV] P despachada [MW]  \
0                 NaN                    NaN               NaN   
1                 NaN                    NaN               NaN   
2             

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,CCSS_AMARIA_TRANSELEC,-,-,NaN,Condensador Sincronico,1,LF7,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0,0,15,15.0,92.0,1,1
1,CCSS_LIKANANTAI_TRANSELEC,-,-,NaN,Condensador Sincronico,1,LF6,00-Norte Grande,NaN,NaN,NaN,PV,92.0,0,0,15,15.0,92.0,1,1
2,CCSS_ILLAPA_ALUPAR,-,-,NaN,Condensador Sincronico,1,LF4,01-Atacama,NaN,NaN,NaN,PQ,270.0,0,0,15,15.0,270.0,1,1
3,CCSS_AMARIA_ALUPAR,-,-,NaN,Condensador Sincronico,1,LF5,00-Norte Grande,NaN,NaN,NaN,PV,360.0,0,0,15,15.0,360.0,1,1
4,CS_U15,-,-,NaN,Condensador Sincronico,1,LF5,00-Norte Grande,NaN,NaN,NaN,PV,147.1,0,0,15,13.8,147.1,1,1


# Uniendo Data

In [31]:
df_concatenado = pd.concat([datos_PV, datos_WP, datos_SG,datos_PMGD,datos_PV_OG,datos_SG_OG,datos_CCSS ], axis=0, ignore_index=True)

#-------- redondeo columnas 
df_concatenado['P_set [MW]'] = df_concatenado['P_set [MW]'].astype(float).round(2)
df_concatenado['Q_set [Mvar]'] = df_concatenado['Q_set [Mvar]'].astype(float).round(2)
df_concatenado['Sbase [MVA]'] = df_concatenado['Sbase [MVA]'].astype(float).round(2)
df_concatenado['Potencia Nominal [MVA]'] = df_concatenado['Potencia Nominal [MVA]'].astype(float).round(2)

df_concatenado
#df_concatenado

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,110.00,67.87,-16.10,NaN,220.0,110.00,66.0,66.0
1,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,94.38,58.23,-18.88,NaN,220.0,94.38,26.0,26.0
2,PFV_EL_ROMERO,PFV_EL_ROMERO,NaN,PFV EL ROMERO,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,206.89,127.65,-41.38,NaN,220.0,206.89,190.0,190.0
3,PFV_VALLE_ESCONDIDO,PFV_VALLE_ESCONDIDO,NaN,PFV VALLE ESCONDIDO,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,106.69,65.83,-10.67,NaN,220.0,106.69,64.0,64.0
4,PFV_SAN_ANDRES,PFV_SAN_ANDRES,NaN,PFV SAN ANDRES,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,50.58,13.56,0.00,NaN,220.0,50.58,35.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,CCSS_AMARIA_TRANSELEC,-,-,NaN,Condensador Sincronico,1,LF7,00-Norte Grande,NaN,NaN,NaN,PV,92.00,0.00,0.00,15,15.0,92.00,1,1
335,CCSS_LIKANANTAI_TRANSELEC,-,-,NaN,Condensador Sincronico,1,LF6,00-Norte Grande,NaN,NaN,NaN,PV,92.00,0.00,0.00,15,15.0,92.00,1,1
336,CCSS_ILLAPA_ALUPAR,-,-,NaN,Condensador Sincronico,1,LF4,01-Atacama,NaN,NaN,NaN,PQ,270.00,0.00,0.00,15,15.0,270.00,1,1
337,CCSS_AMARIA_ALUPAR,-,-,NaN,Condensador Sincronico,1,LF5,00-Norte Grande,NaN,NaN,NaN,PV,360.00,0.00,0.00,15,15.0,360.00,1,1


# UNO DATA DE RESULTADOS

In [2]:
# uno la data de los resultados 
c=-1
N_tabla= len(Tabla_LF_GEN)
df = df_concatenado

for name1 in df_concatenado["Name1"]:
    c= c+1
    df = Tabla_LF_GEN 
    Tipo = df_concatenado.at[c,"Type"]  
    status = df_concatenado.at[c,"EMTP Status"]  
    if Tipo =="Parque Fotovoltaico":
        for i in range(N_tabla):
            if name1 == Tabla_LF_GEN.at[i,"Name1"] and status == 1:
                df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[i,"Tensión en Bornes [kV]"]
                df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[i,"Potencia Activa [MW]"]
                df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[i,"Potencia Reactiva [Mvar]"]
            else:
                continue
                   
    if Tipo =="Generador Sincrónico":
        nameLF=df_concatenado.at[c,"EMTP Load Flow Component"]
        for i in range(N_tabla):
            if nameLF == Tabla_LF_GEN.at[i,"NameLF"] and name1 ==Tabla_LF_GEN.at[i,"Name1"]:   
                df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[i,"Tensión en Bornes [kV]"]
                df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[i,"Potencia Activa [MW]"]
                df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[i,"Potencia Reactiva [Mvar]"]
            else : 
                continue        
    if Tipo =="Parque Eólico":
        for i in range(N_tabla):
            if name1 == Tabla_LF_GEN.at[i,"Name1"]:
                df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[i,"Tensión en Bornes [kV]"]
                df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[i,"Potencia Activa [MW]"]
                df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[i,"Potencia Reactiva [Mvar]"]     
            else : 
                continue      

    
      
        #df_concatenado.at[c,"Tensión en Bornes [kV]"] = Tabla_LF_GEN.at[fila,"Tensión en Bornes [kV]"]
        #df_concatenado.at[c,"P despachada [MW]"] = Tabla_LF_GEN.at[fila,"Potencia Activa [MW]"]
        #df_concatenado.at[c,"Q despachada [Mvar]"] = Tabla_LF_GEN.at[fila,"Potencia Reactiva [Mvar]"]


pd.set_option('display.max_rows', 250)
df_concatenado 

NameError: name 'Tabla_LF_GEN' is not defined

# AGREGO GFM y STATCOM 

In [33]:
GFM_STAT = pd.read_excel('Plantas.xlsx',sheet_name="GFM_STAT" )
data_pre_final = df_concatenado
data_final = pd.concat([data_pre_final,GFM_STAT ], axis=0, ignore_index=True)


# Agrego BESS

In [34]:
BESS = pd.read_excel('Plantas.xlsx',sheet_name="BESS" )
data_f = data_final
data_final2 = pd.concat([data_f,BESS ], axis=0, ignore_index=True)
pd.set_option('display.max_rows', 370)
data_final2

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
0,PFV_EL_PELICANO,PFV_EL_PELICANO,NaN,NaN,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,110.00,67.87,-16.10,NaN,220.0,110.00,66.0,66.0
1,PFV_LA_HUELLA,PFV_LA_HUELLA,NaN,PFV LA HUELLA,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,94.38,58.23,-18.88,NaN,220.0,94.38,26.0,26.0
2,PFV_EL_ROMERO,PFV_EL_ROMERO,NaN,PFV EL ROMERO,Parque Fotovoltaico,1,NaN,02-Coquimbo,NaN,NaN,NaN,NaN,206.89,127.65,-41.38,NaN,220.0,206.89,190.0,190.0
3,PFV_VALLE_ESCONDIDO,PFV_VALLE_ESCONDIDO,NaN,PFV VALLE ESCONDIDO,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,106.69,65.83,-10.67,NaN,220.0,106.69,64.0,64.0
4,PFV_SAN_ANDRES,PFV_SAN_ANDRES,NaN,PFV SAN ANDRES,Parque Fotovoltaico,1,NaN,01-Atacama,NaN,NaN,NaN,NaN,50.58,13.56,0.00,NaN,220.0,50.58,35.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,BESS_6H_I_1,NaN,NaN,NaN,BESS,1,NaN,NaN,NaN,NaN,NaN,NaN,510.00,-204.00,0.00,NaN,220,510.00,510,510
376,BESS_6H_I_2,NaN,NaN,NaN,BESS,0,NaN,NaN,NaN,NaN,NaN,NaN,90.00,0.00,0.00,NaN,220,90.00,90,90
377,BESS_6H_II_12,NaN,NaN,NaN,BESS,0,NaN,NaN,NaN,NaN,NaN,NaN,600.00,0.00,0.00,NaN,220,600.00,600,600
378,BESS_6H_III_1,NaN,NaN,NaN,BESS,0,NaN,NaN,NaN,NaN,NaN,NaN,200.00,0.00,0.00,NaN,220,200.00,200,200


In [35]:
# Agregando plantas sin paginas de power factory 

Nombres = ['PE_Lomas_de_TalTal_OG', "Cochrane_U2","PMGD_PFV_Lagunas220","PMGD_PFV_PAlmonte110","PFV_Willka2_New"]
PF = ["01-Atacama","00-Norte Grande","00-Norte Grande","00-Norte Grande","00-Norte Grande"]
df = data_final2
# Filtrar las filas que contengan el nombre
c=-1

for names in data_final2["Name1"]:
    c=c+1
    pf = -1
    for nombres in Nombres:
        pf = pf +1
        if names == nombres:
            print(c)
            data_final2.at[c,"Página PowerFactory"]= PF[pf]

50
98
222
278
299


In [58]:
aux = data_final2[ ((data_final2['Type'] == 'Generador Sincrónico') | (data_final2['Type'] == 'Condensador Sincronico'))
           & (data_final2['EMTP Status'] == "1")]
aux

,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tensión en Bornes [kV],P despachada [MW],Q despachada [Mvar],Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],Nº_unid_serv,Nº_unid
337,CCSS_U15,-,-,NaN,Condensador Sincronico,1,LF5,NaN,NaN,NaN,NaN,PV,147.1,0.0,0.0,15,13.8,147.1,1,1


# INERCIAs

In [36]:
Data_SG = data_final2[((data_final2['Type'] == 'Generador Sincrónico') | (data_final2['Type'] == 'Condensador Sincronico')|
 (data_final2['Name1']=="CS_U15") )  & (data_final2['EMTP Status'] == 1)]


Data_SG2= Data_SG.reset_index()
Data_SG2 = Data_SG2.drop(["index","Tensión en Bornes [kV]","P despachada [MW]","Q despachada [Mvar]"
                    ,"Nº_unid_serv","Nº_unid"],axis=1)


c=-1
N=len(df_allSG)

for name3 in Data_SG2["Name3"]:
    c=c+1
    for i in range(0,N):
        if name3 == df_allSG.at[i,"Name_of_SG"]:
             Data_SG2.at[c,"H [s]"]= df_allSG.at[i+18,"Value"]



# ------- Adicional 1: Agrego H de los CCSS
             
CCSS= {"Name" : ["CCSS_AMARIA_TRANSELEC", "CCSS_LIKANANTAI_TRANSELEC","CCSS_ILLAPA_ALUPAR"	,"CCSS_AMARIA_ALUPAR","CCSS_U15"],
        "H" : [10,10,5,5,5.04]	}             
CCSS = pd.DataFrame(CCSS)        
             


c=-1
for name in CCSS["Name"]:
    c=c+1
    H= CCSS.at[c,"H"]
    Data_SG2.loc[Data_SG2['Name1'] == name, 'H [s]'] = H

 

#--------------------------------
Data_SG2["H [GVAs]"]= Data_SG2["H [s]"]*Data_SG2["Sbase [MVA]"]*0.001 
Htotal = Data_SG2["H [GVAs]"].sum()
Htotal_NG= Data_SG2[Data_SG2["Página PowerFactory"]=="00-Norte Grande"]["H [GVAs]"].sum()
print(Htotal,Htotal_NG)
Data_SG2

28.691869800000003 6.05332


,Name1,Name2,Name3,Nombre PowerFactory,Type,EMTP Status,EMTP Load Flow Component,Página PowerFactory,Tipo de Barra Flujo de Carga,Sbase [MVA],P_set [MW],Q_set [Mvar],V_set [kV],Tensión Nominal [kV],Potencia Nominal [MVA],H [s],H [GVAs]
0,Central_San_Isidro,San_Isidro_U2_TV,San_Isidro_U2_TV,TER SAN ISIDRO II CC1-TV,Generador Sincrónico,1,LF_SISIDRO_U2_TV,06-Troncal_Qui-Cha,PV,163.00,64.00,480.00,13.2,13.2,163.00,7.920,1.290960
1,Central_San_Isidro,San_Isidro_U2_TG,San_Isidro_U2_TG,TER SAN ISIDRO II CC1-TG,Generador Sincrónico,1,LF_SISIDRO_U2_TG,06-Troncal_Qui-Cha,PV,305.00,105.00,480.00,15.0,15.0,305.00,7.000,2.135000
2,Central_Nehuenco,Nehuenco_U2_TG,Nehuenco_U2_TG,TER NEHUENCO II CC1-TG,Generador Sincrónico,1,LF_NEHU_U2_TG,06-Troncal_Qui-Cha,PV,300.00,110.00,480.00,15.75,15.75,300.00,4.600,1.380000
3,Central_Nehuenco,Nehuenco_U2_TV,Nehuenco_U2_TV,TER NEHUENCO II CC1-TV,Generador Sincrónico,1,LF_NEHU_U2_TV,06-Troncal_Qui-Cha,PV,164.00,85.00,480.00,15.75,15.75,164.00,6.270,1.028280
4,Central_Hornitos,Hornitos_U1,Hornitos_U1,HP HORNITOS U1,Generador Sincrónico,1,LF_HORNIT_U1,03-Chilquinta-Aconcagua,PQ,59.80,22.60,-7.90,11,11.0,59.80,2.700,0.161460
5,Los_Quilos_Juncal,Los_Quillos,LosQuilos,HP LOS QUILOS U1-U3,Generador Sincrónico,1,LF_quilos,03-Chilquinta-Aconcagua,PQ,38.80,32.40,-2.20,6.6,6.6,38.80,1.600,0.062080
6,Los_Quilos_Juncal,Blanco,Blanco,HP BLANCO U1,Generador Sincrónico,1,LF_BLANCO,03-Chilquinta-Aconcagua,PV,56.00,49.50,480.00,11.9,12.0,56.00,2.895,0.162120
7,Los_Quilos_Juncal,Juncal,Juncal,HP JUNCAL U1,Generador Sincrónico,1,LF_JUNCAL,03-Chilquinta-Aconcagua,PV,30.00,18.90,480.00,11.8,12.0,30.00,2.650,0.079500
8,Central_Alfalfal,Alfalfal_U1,Alfalfal_U1,HP ALFALFAL U1,Generador Sincrónico,1,LF_ALFAL_U1,04-Enel Distribución,PV,95.00,53.46,480.00,12.12,12.0,95.00,2.650,0.251750
9,Central_Alfalfal,Alfalfal_U2,Alfalfal_U2,HP ALFALFAL U2,Generador Sincrónico,1,LF_ALFAL_U2,04-Enel Distribución,PV,95.00,53.46,480.00,12.12,12.0,95.00,2.650,0.251750


# ESCRibo excel 

In [37]:

with pd.ExcelWriter('Resultados/Data_SEN_Final_2030Previo.xlsx') as writer:

    data_final2.to_excel(writer, sheet_name='Data',index=False)
    #datos_load.to_excel(writer, sheet_name='Load',index=False)
    
    #df_nuevo.to_excel(writer, sheet_name='Data',index=False)
 

In [69]:
with pd.ExcelWriter('Resultados/DataSG_2028_2024_10_29_0930.xlsx') as writer:
    Data_SG2.to_excel(writer, sheet_name='SG',index=False)